In [1]:
import os
import sys

from snowflake.snowpark.types import StringType
from snowflake.snowpark.functions import udf


from snowflake.snowpark import Session
import pandas as pd
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)



/usr/local/lib/python3.10/dist-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1.dev0+ga6eabc2b.d20230609), please install a version that adheres to: 'pyarrow; extra == "pandas"'
  warn_incompatible_dep(


In [3]:
# Unpack env vars
API_BASE = "http://localhost:8000/v1/completions"
SNOW_ROLE = os.environ["SNOW_ROLE"]
SNOW_WAREHOUSE = os.environ["SNOW_WAREHOUSE"]
SNOW_DATABASE = os.environ["SNOW_DATABASE"]
SNOW_SCHEMA = os.environ["SNOW_SCHEMA"]
SNOW_TABLE = "MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO" #os.environ["SNOW_TABLE"]
SNOW_COLUMN = 'ID' # os.environ['SNOW_COLUMN']

table_sample = '' #int(os.environ.get('table_sample'))
#n = int(os.environ['responses'])
temperature = float(os.environ['temperature'])
max_tokens = int(os.environ['max_tokens'])

# Establish snowpark session
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': SNOW_ROLE,
    'warehouse': SNOW_WAREHOUSE,
    'database': SNOW_DATABASE,
    'schema': SNOW_SCHEMA,
     'session_parameters':{
            'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
        }
}


session = Session.builder.configs(connection_params).create()
token = get_token()



In [6]:
DESTINATION_TABLE = "MISTRAL_VLLM_DB.PUBLIC.PRODUCT_DESCRIPTIONS"
n=1 

@udf(name="build_prompts", is_permanent=True, stage_location="@files", replace=True, session=session)
def build_prompt(name: str, feature_1: str, feature_2: str, target_customer1: str,feature_3: str) -> str:
    PROMPT_TEMPLATE = '''Here is a catchy product description for a {0} that highlights {1} and {2}. Personalize the description for {3} interested in {4}:'''
    return PROMPT_TEMPLATE.format(name, feature_1, feature_2, target_customer1,feature_3)

def get_df(table_name, n = None):
    
    if n:
        df = session.table(table_name).sample(n=n)
    else:
        df = session.table(table_name)
    df_new= df.select(df["id"],build_prompt(df["name"], df["feature_1"], df["feature_2"], df["target_customer1"], df["feature_3"]).as_("prompt"))
    return df_new.select("id", "prompt")
   
df = get_df("MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO",
            table_sample).select("prompt").collect()
prompt_list=pd.DataFrame(df).values.tolist()
print(prompt_list)
outputs = [client.completions.create(model="mistralai/Mistral-7B-Instruct-v0.1",
                                      prompt=p, max_tokens=200, temperature=0.5) for p in prompt_list]
 
    # Print the outputs.
for output in outputs:
        generated_text = output.choices[0].text
        print(f"Generated text: {generated_text!r}")
    #outputs = llm.generate(prompts, sampling_params)
#completion = client.completions.create(model="mistralai/Mistral-7B-Instruct-v0.1",
                                     # prompt="San Francisco is a")

[['Here is a catchy product description for a EJOffice Standup Desk that highlights ergonomic and affordable. Personalize the description for remote office employees interested in made of wood:']]
Generated text: '\n\n"Ergonomic and affordable, our EJOffice Standup Desk is the perfect choice for remote office employees looking to optimize their workspace. Made of high-quality wood, this desk provides a sturdy and stylish surface for your computer, keyboard, and other essentials. With its adjustable height, you can easily switch between sitting and standing positions to reduce strain on your back and improve your posture. Plus, with its sleek design, this desk is sure to fit seamlessly into any home office. Upgrade your workspace and improve your productivity with our EJOffice Standup Desk."'
